In [3]:
import pandas as pd
import plotly.express as px

# Load correlation data
df_corr = pd.read_csv("Monthly_Correlation.csv", sep=";", index_col=0)

# Shorten stock names for better visualization
df_corr.columns = [col[:10] for col in df_corr.columns]  # Truncate long titles
df_corr.index = [idx[:10] for idx in df_corr.index]

# Create an interactive heatmap
fig = px.imshow(
    df_corr,
    color_continuous_scale="RdBu_r",
    labels=dict(color="Correlation"),
    title="Stock Correlation Heatmap",
)

fig.update_layout(
    autosize=False,
    width=900,
    height=700,
    xaxis=dict(tickangle=-45),  # Rotate stock names for readability
    yaxis=dict(automargin=True),
)

fig.show()



In [7]:
# Load CSV with correct separator
df_corr = pd.read_csv("Monthly_Correlation.csv", sep=";", index_col=0)  # Set first column as index (stock names)

# Convert all numeric values to float (excluding first column if needed)
df_corr = df_corr.apply(pd.to_numeric, errors='coerce')

# Fill NaN values with 0
df_corr = df_corr.fillna(0)

# 🔹 **Filter data from 2024** (assuming the first column is 'Date' and formatted as datetime)

df_corr = df_corr[df_corr.index >= "2024-01-01"]

# 🔹 **Split stocks into 2 groups**  
stocks_list = df_corr.columns.to_list()
half = len(stocks_list) // 2

df_corr_part1 = df_corr[stocks_list[:half]]  # First half
df_corr_part2 = df_corr[stocks_list[half:]]  # Second half

# 🔹 **Function to Create a Heatmap**  
def create_heatmap(df_corr_subset, title):
    fig = px.imshow(
        df_corr_subset,
        color_continuous_scale="RdBu_r",
        zmin=-1, zmax=1,
        labels=dict(color="Correlation"),
        title=title,
    )
    fig.update_layout(
        autosize=False,
        width=900,
        height=700,
        xaxis=dict(tickangle=-45),
        yaxis=dict(automargin=True),
    )
    fig.show()

# 🔹 **Generate Two Heatmaps Using the CSV Data**
create_heatmap(df_corr_part1, "Stock Correlation Heatmap (Group 1)")
create_heatmap(df_corr_part2, "Stock Correlation Heatmap (Group 2)")



In [14]:
import pandas as pd
import plotly.express as px

# 🔹 **Step 1: Load the merged file**
df = pd.read_csv("Final_Merged_Data.csv", sep=",", parse_dates=["Fecha"])

# 🔹 **Step 2: Filter only the 'close_price' columns**
close_cols = [col for col in df.columns if "cierre" in col.lower() or col == 'COLCAP_Valor hoy']
df_close = df[["Fecha"] + close_cols].copy()  # Keep 'Date' column for reference

# Ensure 'Date' column is in datetime format
df_close["Fecha"] = pd.to_datetime(df_close["Fecha"])

# Create 'YearMonth' column for grouping
df_close["YearMonth"] = df_close["Fecha"].dt.to_period("M")

# Drop 'Date' and 'YearMonth' before correlation calculations
df_corr = df_close.drop(columns=["Fecha", "YearMonth"])

# Compute monthly correlation and keep it as a DataFrame
monthly_corr = df_close.groupby("YearMonth").corr()

# Handle missing data by forward-filling, then back-filling
df_close.fillna(method='ffill', inplace=True)  # Fill forward
df_close.fillna(method='bfill', inplace=True)  # Fill backward

# 🔹 **Step 3: Set the Target Stock (Update this with your actual target stock)**
target_stock = "COLCAP_Valor hoy"  # Change to your actual stock column name

# Ensure the target stock is in the dataframe
if target_stock not in df_close.columns:
    raise ValueError(f"Target stock '{target_stock}' not found in DataFrame columns!")

# 🔹 **Step 4: Compute correlations only for the target stock**  
df_corr = df_close.corr()[[target_stock]]
# historical_corr = df_corr.corr()[target_stock].drop(target_stock, errors="ignore")
df_corr = df_corr.reset_index().rename(columns={"index": "Stock", target_stock: "Correlation"})

# 🔹 **Step 5: Create Heatmap**
fig = px.imshow(
    df_corr, 
    color_continuous_scale="RdBu_r",
    zmin=-1, zmax=1,
    labels=dict(color="Correlation"),
    title=f"Correlation Heatmap - {target_stock} vs Other Stocks"
)

fig.update_layout(
    autosize=False,
    width=900,
    height=700,
    xaxis=dict(tickangle=-45),
    yaxis=dict(automargin=True),
)

fig.show()


C:\Users\Admin\AppData\Local\Temp\ipykernel_13204\2704728081.py:24: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.

C:\Users\Admin\AppData\Local\Temp\ipykernel_13204\2704728081.py:25: FutureWarning:

DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.



TypeError: float() argument must be a string or a real number, not 'Period'